#                      -- DESASTER --
Simulating household reconstruction with Discrete Event Simulation 

Requires python 3.4

In [1]:
#stdlib and 3rd Party imports
import sys, datetime, os
import numpy as np
import pandas as pd
import simpy
from simpy.util import start_delayed

#add path to desaster module, later we'll install this into site-packages so we shouldn't need to do this
sys.path.append("C:\\Users\\Derek\\desktop\\desaster")
#import desaster files
from desaster import entities, capitals, request, io, movement, search, rebuild


Here we're importing the modules and setting up the stuff

## Load input files for the scenario

In [2]:
#scenario_file = '../inputs/scenario_test1.xlsx'
scenario_file = "../inputs/household_inputs.xlsx"

# Create Pandas dataframe of attribute data for all households to be modeled in the simulation
# required column names, exactly as written: Name , Savings , Insurance
households_df = pd.read_excel(scenario_file, sheetname='households')

# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
housing_stock_df = pd.read_excel(scenario_file, sheetname='housing_stock')

# Set input data for all human capital types, as dict or Pandas Series
# .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]

# Set input data for all financial capital types, as dict or Pandas Series
financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]

### Randomize households and reset index

In [3]:
households_df = households_df.sample(frac=1).reset_index(drop=True)
#takes a random sample, frac is a fraction to sample (1 means take a 100% sample), 
#reset index drops the old scrambled index and puts in a fresh ascending count
households_df.head()

,ID Number,Name,Value,Area,Content_Value,Damage State,Latitude,Longitude,Parcel_ID,Occupancy,Has Insurance,Savings,Insurance
0,US002221,NANA AMATO,28100.0,1228.0,14050.0,Extensive,46.439506,-124.053506,11111632033,Mobile Home,0,2580.981012,0
1,US002058,ARLETTA SHELL,31300.0,1344.0,15650.0,Extensive,46.447120,-124.037207,74071020001,Mobile Home,0,2690.124595,0
2,US001557,JACINDA KUNTZ,19800.0,784.0,9900.0,Extensive,46.468937,-124.036136,74061000021,Mobile Home,0,2252.190562,0
3,US000898,LAURALEE LOMELI,10800.0,505.0,5400.0,Slight,46.467751,-124.054920,11110432092,Single Family Dwelling,0,1826.933575,0
4,US002240,LAVON GARY,49600.0,1782.0,24800.0,Extensive,46.400808,-124.024602,11113412044,Mobile Home,0,3319.106831,0


# Initiate Simulation

In [4]:
simulation = simpy.Environment() 

In [5]:
write_story = True #do we want the story of each household?


In [6]:
human_cap_data

Type
Contractors             40
Engineers                1
FEMA Processors         40
Inspectors             100
Insurance Adjusters     40
Loan Processors         40
Permit Processors      100
Name: Quantity, dtype: int64

In [7]:
human_cap_data["Contractors"] = 100
human_cap_data["Engineers"] = 100
human_cap_data["FEMA Processors"]= 100
human_cap_data["Inspectors"] = 100
human_cap_data["Insurance Adjusters"] = 100
human_cap_data["Loan Processors"] = 100
human_cap_data["Permit Processors"] = 100


In [8]:
financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) #resource

human_capital = capitals.HumanCapital(simulation, human_cap_data) #resource

households = entities.importHouseholds(simulation, households_df, write_story) #entity object container

housing_stock = capitals.importHousingStock(simulation, housing_stock_df) #available housing

### Write a function that controls the flow for each household.

In [9]:
def master_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 1000  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
                return
        
        # If home is completely damaged, search for a new home to purchase.
        if entity.residence.damage_state == 'Complete':
            
            home_patience = 550  # days until give up the search for a new home

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, housing_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))

In [10]:
# Initiate a master process for each household to be modeled in the simulation
for i in range(len(households)):
    simulation.process(master_process(simulation, human_capital, financial_capital, households[i], write_story))

In [11]:
undamaged_housing = 0
for i in housing_stock.items:
    if i.damage_state == "None":
        undamaged_housing += 1
        
print (undamaged_housing)

16


## Rebuild the housing stock

In [12]:
# Do inspections on all of the vacant homes in the housing stock
for home in housing_stock.items:
    simulation.process(request.inspection(simulation, human_capital, home))

# Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# with probability = fix_probability
fix_probability = 1.0
fix_schedule = 100

start_delayed(simulation, rebuild.stock(simulation, housing_stock, fix_probability), fix_schedule)

<Process(starter) object at 0x9a8a570>

# Parameters for the run

In [13]:
human_cap_data

Type
Contractors            100
Engineers              100
FEMA Processors        100
Inspectors             100
Insurance Adjusters    100
Loan Processors        100
Permit Processors      100
Name: Quantity, dtype: int64

## Run the model

In [14]:
#Reload building material at a preordained time
start_delayed(simulation, capitals.reloadBuildingMaterial(simulation, financial_capital.building_materials, amount = 100000000), 100)

simulation.run()

312 homes in the vacant building stock were fixed on day 100.


## Outputs to verify model ran correctly

In [15]:
num_undamaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_gave_up_home_search = 0

for household in households:
    if household.money_search_start == None: num_undamaged += 1
    if household.home_get is not None: num_rebuilt += 1
    if household.gave_up_money_search: num_gave_up_money_search += 1   
    if household.home_search_stop is not None:
        if household.home_search_stop > 0.0: num_relocated += 1
    if household.gave_up_home_search: num_gave_up_home_search += 1
        
print('{0} out of {1} households suffered no damage to their homes.\n'.format(num_undamaged, len(households)),
      '{0} out of {1} households rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(households)),
        '{0} out of {1} households gave up searching for money.\n'.format(num_gave_up_money_search, len(households)),
        '{0} out of {1} households decided to find a new home.\n'.format(num_relocated, len(households)),
        '{0} out of {1} households gave up searching for a home.'.format(num_gave_up_home_search, len(households))
      )

46 out of 2860 households suffered no damage to their homes.
 2759 out of 2860 households rebuilt or repaired their damaged home.
 0 out of 2860 households gave up searching for money.
 57 out of 2860 households decided to find a new home.
 1 out of 2860 households gave up searching for a home.


In [16]:
vars(households[1258])

{'assessment_get': 140.74541175392932,
 'assessment_put': 19.674387314689877,
 'assistance_get': None,
 'assistance_payout': 0,
 'assistance_put': None,
 'assistance_request': 0,
 'claim_get': None,
 'claim_payout': 0,
 'claim_put': None,
 'gave_up_home_search': False,
 'gave_up_money_search': False,
 'home_get': 222.80407163686479,
 'home_put': 220.8040716368648,
 'home_search_start': None,
 'home_search_stop': None,
 'household': ID Number                      US000143
 Name                        LAURE GEARY
 Value                             26900
 Area                             803.42
 Content_Value                     13450
 Damage State                     Slight
 Latitude                        46.7182
 Longitude                       -124.01
 Parcel_ID                   78016011002
 Occupancy        Single Family Dwelling
 Has Insurance                         0
 Savings                         2553.45
 Insurance                             0
 Name: 1258, dtype: object,
 'in

# MAKE A NEW DATAFRAME FOR EXPORT

In [17]:
#fills the empty dataframe we made above for the output. incredibly badly written
a = list(vars(households[4]).keys()) #gets all potential column names
a.remove("household");a.remove("residence") #remove the stuff we don't want
a.append("latitude");a.append("longitude") #add stuff we do want
df = pd.DataFrame(columns=a)
iters = 0
att_itter = 0
new_column={}
log = []
for i in households: #loop through all entities
    i.latitude = i.household["Latitude"] #extracting lat and long from the residence object
    i.longitude = i.household["Longitude"]
    for att in a: #loop through the attributes in our list of column names we want
        try:
            new_column[att] = i.__getattribute__(att) #set the b dictionary
            #mydata[att]= i.__getattribute__(att)
            
        except ValueError:
            new_column[att] = np.nan
        except AttributeError as e:
            new_column[att] = np.nan
            log.append("Household {0} had an attr error, {1}".format(i.name, e))
        finally:
            att_itter += 1
    mydata=pd.DataFrame([new_column]) #this turns our newly made column into a database where it can be combined with the df

    df = df.append(mydata, ignore_index=True)

    iters += 1
    

print(iters)
print(att_itter)


2860
88660


In [18]:
df.head()
output_path = "../outputs/Output{}.csv".format(str(datetime.date.today()))
df.to_csv(output_path)

In [19]:
#TODO Go in and change the "story" variable to format the strings nice. change it in the root code so its not janky
#and lines up correctly.

In [20]:
df["wait"]=df["inspection_get"]-df["inspection_put"]

In [22]:
df.describe()

,assessment_get,assessment_put,assistance_payout,assistance_request,claim_payout,inspection_get,inspection_put,insurance,latitude,loan_amount,longitude,money_to_rebuild,savings,wait
count,2759.000000,2759.000000,2860.000000,2860.000000,2860.000000,2860.000000,2860.0,2.860000e+03,2860.000000,2860.0,2860.000000,2860.000000,2860.000000,2860.000000
mean,385.207692,128.359247,8794.097150,8794.097150,1213.041608,21.249714,0.0,1.545087e+04,46.461438,0.0,-124.000764,13094.761009,3087.622251,21.249714
std,208.606004,116.269929,12721.127994,12721.127994,10357.058099,6.772652,0.0,6.817419e+04,0.122050,0.0,0.120577,16131.990811,1013.926631,6.772652
min,36.018837,9.837194,0.000000,0.000000,0.000000,9.837194,0.0,0.000000e+00,46.242514,0.0,-124.097405,10.000000,0.000000,9.837194
25%,194.748231,26.885387,0.000000,0.000000,0.000000,15.575557,0.0,0.000000e+00,46.361818,0.0,-124.053941,3570.878534,2385.235187,15.575557
50%,378.019737,72.696264,3637.500000,3637.500000,0.000000,21.313920,0.0,0.000000e+00,46.452576,0.0,-124.048717,7775.199086,3098.741734,21.313920
75%,561.291242,217.454640,13921.875000,13921.875000,0.000000,27.052283,0.0,0.000000e+00,46.493837,0.0,-124.035370,17617.255808,3766.548563,27.052283
max,744.562747,382.073019,131000.000000,131000.000000,327700.000000,32.790646,0.0,2.058900e+06,46.793976,0.0,-123.364914,333296.663171,6296.592929,32.790646


In [ ]:
df.columns


In [ ]:
df.describe()